In [1]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

class Instruction:
    """
    operation_flag: choice of [
        "q" (question),
        "o" (operation),
        "r" (response)
    ]
    
    content: str
    """
    def __init__(self, instruction):
        self.operation_flag:str = instruction[0]
        self.content:str = str(instruction[1])
        
        assert self.operation_flag in ["q", "o", "r"], instruction
    
    def __repr__(self) -> str:
        return f"Instruction(operation_flag={self.operation_flag}, content={self.content})"

class InputToInstruction:
    # model_id = 'sh2orc/Llama-3.1-Korean-8B-Instruct'
    model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'

    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        cache_dir="/model"
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        cache_dir="/model"
    )

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
def execute(PROMPT, input_text:str) -> list[Instruction]:
    print(f"input_text: {input_text}")
    messages = [
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{input_text}"}
    ]

    input_ids = InputToInstruction.tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(InputToInstruction.model.device)

    # https://hipster4020.tistory.com/194
    outputs = InputToInstruction.model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=InputToInstruction.terminators,
        pad_token_id=InputToInstruction.tokenizer.eos_token_id,
        
        # do_sample=False,
        # # num_beams=1,
        # temperature=None,
        
        do_sample=True,
        temperature=0.8,
        top_k=5,
        top_p=0.9;
        # penalty_alpha=0.6,
        repetition_penalty = 1.0,
    )

    result_string:str = InputToInstruction.tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    return result_string
    instructions = []
    result_string = result_string.replace("[[", "").replace("\n", "").replace("\t", "").replace('"', "")
    result_string = result_string.split("],")
    
    stop = False
    for instruction in result_string:
        # "또 다른 예시" 방지
        if "]]" in instruction:
            instruction = instruction.split("]]")[0]
            stop = True
        
        instruction = instruction.replace("[", "").replace("]", "")
        instruction = instruction.split(", ")
        instruction[0] = instruction[0].strip()
        instructions.append(Instruction(instruction))
        
        if stop:
            break
    
    return instructions


In [3]:
PROMPT = \
"""너는 훌룡한 AI HVAC 챗봇이야. 거짓 정보들은 말하지 말아줘.
사용자의 입력에 대해 다음을 고려해 출력해줘.
예시를 통해 이해하는것도 좋지만, 조건들을 엄격히 지켜야 함.
좋은 예시들은 비슷하게, 틀린 예시들은 틀린 이유를 참고하여 다르게 작성해야 함.

<사전정보>
위치: '영동일고등학교'.
알려진 공간: [('우리반', '01_IB5'), ('옆반', '01_IB7')]
질문자: 이름:'홍길동', 분류: '손님'
Modality: ['실내온도', '설정온도', '전원', '기타']
현재시간: '2022-09-30 12:00:00'

<출력 형태 및 조건>
출력은 1. '질문 요약 reasoning', 2. '질문 요약', 3. 'Semantic Parsing reasonsing', 4. 'Semantic Parsing'(dict), 5. 'Instruction Set reasoning', 6. 'Instruction Set'(list), 7. 참고로 구성된 dictionary 형태로 제공되어야 함.
1. '질문 요약 reasoning': 사용자의 질문을 요약하는 과정에서 이유를 설명. 쓰여있는 정보를 활용하고, 없으면 추측한다. Semantic parsing보다 추상적이어야 함.
2. '질문 요약': 사용자의 질문을 요약한 내용.
3. 'Semantic Parsing reasonsing': Semantic Parsing을 하는 과정에서 이유를 설명. 이 과정에서 사전정보에 없는 장소나 구체적인 시간/시간기간을 추측해야만 함.
4. 'Semantic Parsing': Semantic Parsing 결과. dict 형태로 구성되며, Temporal, Spatial, Modality, Type/Quantity, Target, Question/Actuation key를 가짐.
5. 'Instruction Set reasoning': Instruction Set을 만드는 과정에서 이유를 설명.
6. 'Instruction Set': Instruction Set 결과. list 형태로 구성되며, 각 원소는 [flag, content, variable]로 구성됨.
7. 참고: 복잡한 질문에 대한 답변 혹은 알수없음/답변불가시 선택적으로 작성할 수 있으며 Semantic parsing과 Instruction set 다음줄에 나와야 하며, 최대한 간결하게 작성해야 함. 신뢰성 있는 정보를 제공해야 함. 예를들어 예시와 조건에 포함되지 않는 추측성 내용은 작성하지 말아야 함.

<Semantic Parsing 조건>
Temporal: 시간 혹은 시간 범위를 나타내는 정보. 사전정보 참고. 대표적 representation과 Timestamp 형식의 튜플로 표현됨. 아무런 정보가 없을 경우 '지금'으로 표현되지만, 정보가 있는 경우 자제해야 함.
Spatial: 공간 혹은 공간 범위를 나타내는 정보. 사전정보 참고. 대표적 representation과 IDU 번호의 튜플로 표현됨. 아무런 정보가 없을 경우 '우리반'으로 표현되지만, 정보가 있는 경우 자제해야 함. 사전정보에 없는 공간을 언급하는 질문은 '답변불가'로 응답해야 함.
Modality: 정보의 형태 혹은 정보의 속성. 사전정보 참고.
Type/Quantity: 정보의 종류 혹은 양. 
Target: 정보의 대상.
Question/Actuation: 질문 혹은 행위.

<Instruction Set 조건>
공통: 'Semantic Parsing'의 정보를 활용하여 질문을 만들어야 함. 알 수 없음이 포함될 수 없다.
q: Query을 나타내는 flag. 두 번째 인자는 Query Generator에서 사용할 예정이며, 세 번째 인자는 query의 결과를 저장할 변수명이다.
o: Operation을 나타내는 flag. 두 번째 인자는 Operation Generator에서 사용할 예정이며, 세 번째 인자는 operation의 결과를 저장할 변수명이다.
r: Response를 나타내는 flag. 두 번째 인자는 Response를 제작하는 데 사용할 변수명이며, 세 번째 인자는 Response의 예시를 나타낸다.

<절대로 하지 말아야 할 것>
출력 앞 뒤에 불필요한 내용이 있거나 dictionary 형태를 지키지 않으면 절대로 안됨(즉, 중괄호가 무조건 있어야함). 특히 '~~질문에 대한 답변은 다음과 같습니다.나 주어진예시를~~'와 같은 문장은 절대로 작성하지 않아야 함.
이미 명시적으로 시간/공간 정보가 input에 있는데, 없다고 잘못 판단하면 안됨.

<예시들과 다른 입력이 들어올 경우>
예시들과 조금 다르더라도 출력의 형식은 무조건 지켜야함.

<좋은 예시1>
입력: 오늘 우리반과 옆반의 온도차이 알려줘.
출력: {
    "질문 요약 reasoning": "시간은 오늘이라고 쓰여 있다. 장소는 우리반과 옆반이라고 쓰여 있다. 온도는 실내온도로 추정된다.",
    "질문 요약": "오늘 우리반과 옆반의 실내온도 차이 알려줘.",
    "Semantic Parsing reasonsing": "오늘은 2022년 9월 30일이며, 우리반은 01_IB5, 옆반은 01_IB7이다."
    "Semantic Parsing": {
        "Temporal": [("오늘", "2022-09-30 00:00:00 ~ 2022-09-30 23:59:59")],
        "Spatial": [("우리반", "01_IB5"), ("옆반", "01_IB7")],
        "Modality": ["실내온도"],
        "Type/Quantity": ["diff"],
        "Target": ["온도"],
        "Question/Actuation": ["알려줘"]
    },
    "Instruction Set reasoning": "온도차이를 알려주기 위해 두 공간의 온도를 먼저 알아야 하며, 이를 뺀 값을 출력해야 한다."
    "Instruction Set": [
        ["q", "{{Temporal_1}} {{Spatial_1}} {{Modality_1}} {{QA_1}}", "V_1"],
        ["q", "{{Temporal_1}} {{Spatial_2}} {{Modality_1}} {{QA_1}}", "V_2"],
        ["o", "V_1 - V_2", "V_3"],
        ["r", "V_3", "예) '우리반이 옆반보다 2도 높아요'"]
    ]
}

<좋은 예시2>
입력: 지난 여름 우리반 평균 설정온도 알려줘.
출력: {
    "질문 요약 reasoning": "시간은 지난 여름라고 쓰여 있다. 지난 여름은 올해 여름이라는 뜻으로 추정된다. 장소는 우리반이라고 쓰여있다.",
    "질문 요약": "올해 여름 우리반 평균 설정온도 알려줘.",
    "Semantic Parsing reasonsing": "올해는 2022년이며, 여름은 6월 1일부터 8월 31일까지이다. 우리반은 01_IB5이다."
    "Semantic Parsing": {
        "Temporal": [("올해 여름", "2022-06-01 00:00:00 ~ 2022-08-31 23:59:59")],
        "Spatial": [("우리반", "01_IB5")],
        "Modality": ["설정온도"],
        "Type/Quantity": ["mean"],
        "Target": ["온도"],
        "Question/Actuation": ["알려줘"]
    },
    "Instruction Set reasoning": "평균 설정온도를 알려주기 위해 기간 내 온도를 구하고 이를 평균내야 한다."
    "Instruction Set": [
        ["q", "{{Temporal_1}} {{Spatial_1}} {{Modality_1}} {{QA_1}}", "V_1"],
        ["o", "np.mean(V_1)", "V_2"],
        ["r", "V_2", "예) '지난 여름 우리반의 평균 설정온도는 26도입니다.'"]
    ],
}

<좋은 예시3>
입력: 이번달 중 우리반 온도가 가장 더운날이 언제야?
출력: {
    "질문 요약 reasoning": "시간은 이번달이라고 쓰여있다. 장소는 우리반이라고 쓰여 있다. 온도는 실내온도로 추정된다. 더운날은 온도가 가장 높은 날을 의미한다."
    "질문 요약": "이번달 우리반 실내온도가 가장 높은 날짜 알려줘."
    "Semantic Parsing reasonsing": "이번달은 9월 1일부터 9월 30일이다. 우리반은 01_IB5이다. 가장 높은 온도가 아닌 그 날짜를 알아야 한다."
    "Semantic Parsing": {
        "Temporal": [("이번달", "2022-09-01 00:00:00 ~ 2022-09-30 23:59:59")],
        "Spatial": [("우리반", "01_IB5")],
        "Modality": ["실내온도"],
        "Type/Quantity": ["argmax"],
        "Target": ["날짜"],
        "Question/Actuation": ["알려줘"]
    },
    "Instruction Set reasoning": "온도가 가장 높은 날짜를 알기 위해 우선 온도를 구하고, 이 중 가장 높은 값을 찾은 뒤 그 날짜를 출력해야 한다."
    "Instruction Set": [
        ["q", "{{Temporal_1}} {{Spatial_1}} {{Modality_1}} {{QA_1}}", "V_1"],
        ["o", "np.argmax(V_1)", "V_2"],
        ["r", "V_2", "예) '이번달 중 우리반의 온도가 가장 더운날은 2022년 9월 15일입니다.'"]
    ]
}

<좋은 예시4>
입력: 우리반과옆반중에온도가더낮은곳은어디야?
출력: {
    "질문 요약 reasoning": "시간은 지금으로 추측한다. 장소는 우리반과 옆반이라고 쓰여있다. 온도는 실내온도로 가정한다. 곳은 장소를 의미한다."
    "질문 요약": "지금 우리반과 옆반 중 실내온도가 더 낮은 장소 알려줘."
    "Semantic Parsing reasonsing": "지금은 2022년 9월 30일 12시 00분이다. 우리반은 01_IB5, 옆반은 01_IB7이다."
    "Semantic Parsing": {
        "Temporal": [("지금", "2022-09-30 12:00:00")],
        "Spatial": [("우리반", "01_IB5"), ("옆반", "01_IB7")],
        "Modality": ["실내온도"],
        "Type/Quantity": ["argmin"],
        "Target": ["장소"],
        "Question/Actuation": ["알려줘"]
    },
    "Instruction Set reasoning": "온도가 더 낮은 장소를 알기 위해 우선 두 공간의 온도를 비교하고, 이를 토대로 더 낮은 장소를 출력해야 한다."
    "Instruction Set": [
        ["q", "{{Temporal_1}} {{Spatial_1}} {{Modality_1}} {{QA_1}}", "V_1"],
        ["q", "{{Temporal_1}} {{Spatial_2}} {{Modality_1}} {{QA_1}}", "V_2"],
        ["o", "{{Spatial_1}} if V_1 > V_2 else {{Spatial_2}}", "V_3"],
        ["r", "V_3", "예) '우리반의 온도가 옆반보다 더 낮아요.'"]
    ]
}

<좋은 예시5>
입력: 에어컨 켜져있어?
출력: {
    "질문 요약 reasoning": "시간은 지금으로 추측한다. 장소는 우리반으로 추측한다. 에어컨의 전원 여부를 알려달라는 의미이다."
    "질문 요약": "지금 우리반 전원 값 알려줘."
    "Semantic Parsing reasonsing": "지금은 2022년 9월 30일 12시 00분이다. 우리반은 01_IB5이다."
    "Semantic Parsing": {
        "Temporal": [("지금", "2022-09-30 12:00:00")],
        "Spatial": [("우리반", "01_IB5")],
        "Modality": ["전원"],
        "Type/Quantity": ["value"],
        "Target": ["전원"],
        "Question/Actuation": ["알려줘"]
    },
    "Instruction Set reasoning": "전원 값을 알기 위해 우선 전원 값을 구하고, 이를 출력해야 한다. 특별한 연산이 필요하지 않다."
    "Instruction Set": [
        ["q", "{{Temporal_1}} {{Spatial_1}} {{Modality_1}} {{QA_1}}", "V_1"],
        ["r", "V_2", "예) '지금 우리반 에어컨은 켜져 있습니다.'"]
    ]
}

<좋은 예시6>
입력: 앞반 지금 전원 켜져있어?
출력: {
    "질문 요약 reasoning": "시간은 지금으로 추측한다. 장소는 앞반으로 쓰여있다. 전원에 대한 정보를 알고 싶어하는 것으로 추정된다."
    "질문 요약": "지금 앞반 전원 값 알려줘."
    "Semantic Parsing reasonsing": "지금은 2022년 9월 30일 12시 00분이다. 앞반은 사전정보에 없는 공간이므로 답변불가로 응답해야 한다."
    "Semantic Parsing": {
        "Temporal": [("지금", "2022-09-30 12:00:00")],
        "Spatial": [("답변불가", None)],
        "Modality": ["전원"],
        "Type/Quantity": ["value"],
        "Target": ["전원"],
        "Question/Actuation": ["알려줘"]
    },
    "Instruction Set reasoning": "답변불가로 응답해야 하므로 Instruction Set이 필요하지 않다. 왜 답변불가인지 설명해야 한다."
    "Instruction Set": [
        ["r", None, "예) '앞반은 사전정보에 없는 공간이므로 답변불가입니다.'"]
    ]
}

<좋은 예시7>
입력: 우리집 지금 전력사용량 얼마야?
출력: {
    "질문 요약 reasoning": "시간은 지금으로 쓰여있다. 장소는 우리집으로 쓰여있다. 전력사용량을 알고 싶은것으로 추정된다."
    "질문 요약": "지금 우리집 전력사용량 알려줘."
    "Semantic Parsing reasonsing": "지금은 2022년 9월 30일 12시 00분이다. 우리집은 사전정보에 없는 공간이므로 답변불가로 응답해야 한다. 전력사용량은 사전정보에 없는 정보이므로 답변불가로 응답해야 한다."
    "Semantic Parsing": {
        "Temporal": [("지금", "2022-09-30 12:00:00")],
        "Spatial": [("답변불가", None)],
        "Modality": ["답변불가"],
        "Type/Quantity": ["value"],
        "Target": ["답변불가"],
        "Question/Actuation": ["알려줘"]
    },
    "Instruction Set reasoning": "답변불가로 응답해야 하므로 Instruction Set이 필요하지 않다. 왜 답변불가인지 설명해야 한다."
    "Instruction Set": [
        ["r", None, "예) '우리집은 사전정보에 없는 공간이고 전력사용량은 사전정보에 없는 정보이므로 답변불가입니다.'"]
    ]
}

<좋은 예시8>
입력: 어제 전원 껐어?
출력: {
    "질문 요약 reasoning": "시간은 어제로 쓰여있다. 장소는 우리반으로 추측한다. 전원값을 알고 싶어하는 것으로 추정된다."
    "질문 요약": "어제 전원 값 알려줘."
    "Semantic Parsing reasonsing": "어제는 2022년 9월 29일이다. 우리반은 01_IB5이다. 에어컨의 전원 여부를 알고 싶어하는 것으로 추정된다."
    "Semantic Parsing": {
        "Temporal": [("어제", "2022-09-29 00:00:00 ~ 2022-09-29 23:59:59")],
        "Spatial": [("우리반", "01_IB5")],
        "Modality": ["전원"],
        "Type/Quantity": ["value"],
        "Target": ["전원"],
        "Question/Actuation": ["알려줘"]
    },
    "Instruction Set reasoning": "어제의 전원 상태를 알기 위해 어제의 전원 값을 구하고, 이를 출력해야 한다."
    "Instruction Set": [
        ["q", "{{Temporal_1}} {{Spatial_1}} {{Modality_1}} {{QA_1}}", "V_1"],
        ["r", "V_2", "예) '어제 우리반 에어컨은 꺼져 있었습니다.'"]
    ]
}

"""

print(execute(PROMPT,"지난 여름 우리반과 옆반의 풍량 비교해줘"))
print(execute(PROMPT,"3일전 전원 껐어?"))
print(execute(PROMPT,"옆반 지금 에어컨 켜져있어?"))
print(execute(PROMPT,"뒷반 여름 평균 설정온도 알려줘."))
print(execute(PROMPT,"내일 우리반과 옆반의 온도차이 알려줘."))
print(execute(PROMPT,"이번달 중 우리반 전원 켜진날이 언제야?"))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


input_text: 지난 여름 우리반과 옆반의 풍량 비교해줘
제가 이해한 바로는, '지난 여름 우리반과 옆반의 풍량 비교'라는 질문은 다음과 같이 분석될 수 있습니다.

1. '질문 요약 reasoning': "시간은 지난 여름이라고 쓰여 있다. 지난 여름은 올해 여름이라는 뜻으로 추정된다. 장소는 우리반과 옆반으로 쓰여 있다."
2. '질문 요약': "올해 여름 우리반과 옆반의 풍량 비교 알려줘."
3. 'Semantic Parsing reasonsing': "올해는 2022년이며, 여름은 6월 1일부터 8월 31일까지이다. 우리반은 01_IB5, 옆반은 01_IB7이다."
4. 'Semantic Parsing': {
    "Temporal": [("올해 여름", "2022-06-01 00:00:00 ~ 2022-08-31 23:59:59")],
    "Spatial": [("우리반", "01_IB5"), ("옆반", "01_IB7")],
    "Modality": ["풍량"],
    "Type/Quantity": ["compare"],
    "Target": ["풍량"],
    "Question/Actuation": ["알려줘"]
}
5. 'Instruction Set reasoning': "풍량을 비교하기 위해 우선 두 공간의 풍량을 구하고, 이를 비교하여 차이를 출력해야 한다."
6. 'Instruction Set': [
    ["q", "{{Temporal_1}} {{Spatial_1}} {{Modality_1}} {{QA_1}}", "V_1"],
    ["q", "{{Temporal_1}} {{Spatial_2}} {{Modality_1}} {{QA_1}}", "V_2"],
    ["o", "V_1 - V_2", "V_3"],
    ["r", "V_3", "예) '우리반과 옆반의 풍량을 비교했더니 우리반은 옆반보다 10m³/h 더 많은데요.'"]
]

따라서, 위와 같은 구조로 답변을 제공할 수 있습니다.
input_text: 3